In [ ]:
````xml
<VSCode.Cell language="python">
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from models import DnCNN
from data_utils import load_dataset
from trainer import Trainer, get_default_config, visualize_results

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
</VSCode.Cell>
<VSCode.Cell language="python">
config = get_default_config('DnCNN')
config.update({
    'epochs': 100,
    'learning_rate': 1e-3,
    'batch_size': 64,
    'dataset': 'mnist',
    'noise_type': 'gaussian',
    'noise_level': 0.2
})

train_loader, val_loader, test_loader, channels = load_dataset(
    config['dataset'], 
    config['batch_size'], 
    config['noise_type'], 
    config['noise_level']
)

print(f"Dataset: {config['dataset']}")
print(f"Channels: {channels}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")
</VSCode.Cell>
<VSCode.Cell language="python">
model = DnCNN(channels=channels, num_layers=17, features=64)
trainer = Trainer(model, train_loader, val_loader, test_loader, device, config)

print("Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
</VSCode.Cell>
<VSCode.Cell language="python">
trainer.train()
</VSCode.Cell>
<VSCode.Cell language="python">
test_loss, test_psnr, test_ssim = trainer.test()
</VSCode.Cell>
<VSCode.Cell language="python">
trainer.plot_metrics()
</VSCode.Cell>
<VSCode.Cell language="python">
visualize_results(model, test_loader, device, num_samples=8)
</VSCode.Cell>
<VSCode.Cell language="python">
noise_levels = [0.1, 0.15, 0.2, 0.25, 0.3]
results = []

for noise_level in noise_levels:
    print(f"\nTesting noise level: {noise_level}")
    _, _, test_loader_noise, _ = load_dataset(
        config['dataset'], 
        config['batch_size'], 
        config['noise_type'], 
        noise_level
    )
    
    model.eval()
    total_psnr = 0
    total_ssim = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in test_loader_noise:
            noisy, clean, _ = batch
            noisy, clean = noisy.to(device), clean.to(device)
            
            output = model(noisy)
            
            from data_utils import calculate_psnr, calculate_ssim
            total_psnr += calculate_psnr(output, clean).item()
            total_ssim += calculate_ssim(output, clean).item()
            num_batches += 1
    
    avg_psnr = total_psnr / num_batches
    avg_ssim = total_ssim / num_batches
    results.append((noise_level, avg_psnr, avg_ssim))
    print(f"PSNR: {avg_psnr:.2f}dB, SSIM: {avg_ssim:.4f}")

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot([r[0] for r in results], [r[1] for r in results], 'o-')
plt.xlabel('Noise Level')
plt.ylabel('PSNR (dB)')
plt.title('DnCNN Performance vs Noise Level')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot([r[0] for r in results], [r[2] for r in results], 'o-')
plt.xlabel('Noise Level')
plt.ylabel('SSIM')
plt.title('DnCNN SSIM vs Noise Level')
plt.grid(True)

plt.tight_layout()
plt.show()
</VSCode.Cell>
````